# Homework 3

In [6]:
from tqdm.auto import tqdm

## Q1. Getting the embeddings model

First, we will get the embeddings model multi-qa-distilbert-cos-v1 from the [Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

What's the first value of the resulting vector?
* **0.07**

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('multi-qa-distilbert-cos-v1')
user_question = "I just discovered the course. Can I still join it?"
v = model.encode(user_question)
len(v)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


768

In [2]:
v[0]

0.078222655

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

What's the shape of X? `(X.shape)`. Include the parantheses.
* **(375, 768)**

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
def filter_documents(doc):
    if doc['course'] == "machine-learning-zoomcamp":
        return True
    return False

documents_iterator = filter(filter_documents, documents)
filtered_documents = list(documents_iterator)
len(filtered_documents)

375

In [7]:
embeddings = []
for doc in tqdm(filtered_documents):
    qa_text = f"{doc['question']} {doc['text']}"
    emb = model.encode(qa_text)
    doc['qa_vector'] = emb.tolist()
    embeddings.append(emb)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:05<00:00,  5.73it/s]


In [8]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix `X` by the vector `v`:

```python
scores = X.dot(v)
```

What's the highest score in the results?
* **0.65**

In [9]:
scores = X.dot(v)
max(scores)

0.6506573

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will use the hitrate metric for evaluation.

Now use the code from the module to calculate the hitrate of `VectorSearchEngine` with `num_results=5`.

What did you get?
* **0.33**

In [10]:
import pandas as pd

class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

def hit_rate(relevance_total):
    count = 0
    for line in relevance_total:
        if True in line:
            count += 1

    return count / len(relevance_total)

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [11]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)

relevance_total = []
for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(v, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:00<00:00, 18305.73it/s]


In [12]:
hit_rate(relevance_total)

0.01366120218579235

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch:
* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)
  
After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [13]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200', request_timeout=60, max_retries=10, retry_on_timeout=True)
es_client.info()

ObjectApiResponse({'name': '38fbeb184f3f', 'cluster_name': 'docker-cluster', 'cluster_uuid': '4gE-8p9lSpe_fzCPNYtLKA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "qa_vector": {
                "type": "dense_vector",
                "dims": 375,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

In [15]:
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'course-questions'})

In [ ]:
for doc in tqdm(filtered_documents):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|                                                                                                                                     | 0/375 [00:00<?, ?it/s]

In [ ]:
result = es_client.search(index=index_name, body=v)
result["hits"]["hits"]

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between the query and ALL the vectors in our database. Usually this is not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster.

Let's evaluate how worse the results are when we switch from exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?
* **0.73**